In [ ]:
from collections import OrderedDict
import json
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import psycopg2

from lib.data_tools import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Taxonomy by product
product groups to be defined by:
- property type
- loan purpose
- loan type
- lien status
- occupancy

Analysis variables to consider:
- action
- purchaser
- amount
- income
- preapproval
- rate spread
- race
- geo (national, MSA, state, tract)

### Create tables by product
- Include analysis variables
- Aggregate from tables

### April Edits
April Edits Tests:
- loan amount percentiles, mean, std dev, skew, kurtosis
- action frequency, percent, cumulative percent
- Race/Action matrix counts
- applicant sex frequency, percent, cumulative percent
- co applicant sex: frequency percent, cumulative percent
- race: frequency, percent, cumulative percent
- ethnicity if race=5: frequency, percent, cumulative percent
- income for action=1: percentiles, mean, stdev, skew, kurtosis
- income for action=3: percentiles, mean, stdev, skew, kurtosis
- income for action=1, loan_type=1: percentiles, mean, stdev, skew, kurtosis
- income for action=1, loan_type=2: percentiles, mean, stdev, skew, kurtosis
- purchaser type for loan_type=1, aciton=1: count, percent
- purchaser type for loan_type>1, aciton=1: count, percent
- rate spread precentiles, mean, stdev, skew, kurtosis
- inst name for lien=1, loantype=1, propertytype=1, spread>=15: frequency, percent
- inst name for lien=1, loantype=1, propertytype=2, spread>=15: frequency, percent
- denial1 if loantype=1, action=3: freq, percent, cumulative percent
- denial1 if loantype=2, action=3: freq, percent, cumulative percent
- denial1 if loanpurpose=3, aciton=3: freq, percent, cumulative percent
- frequency by state
- institutions with >10 single family, site-built, first lien, owner occupied, w/ rate spread >6.5 (hoepa)
- institutions with >10 single family, site-built, first lien, owner occupied, w/ rate spread >8.5 (hoepa)
- agency, rid, name, prior year app count, count state=00 for institutions w >=500 apps with state=00
- institutions with >=250 applications with action<=3 with all applications having a single race
- institutions with >=250 applications with action<=3 with all applications having a single ethnicity
- institutions with >=50 applications with action<=3 with all applications having a single sex
- institutions with >=50 applications with action<=3 with all applications having a single co-app sex or no co-app
- institutions with >=50 applications with action<=3 with >50% applications to same sex couples
- institions with >=100 originatios, where >=75% are home purchase or refi and >75% are junior liens or unsecured
- institutions with >=25 loans and >=50% of loans are over 10 million
- institutions with >=50 home purchase, first-lien owner-occupied loans where income is NA in >25% of loans
- institutions with >=50 home purchase, first-lien owner-occupied loans where income >=1 million in >50% of loans
- institutions with >=50 home purchase, first-lien owner-occupied loans where income < 10k in >50% of loans
- institutions with >=100 FHA first-lien home purchase loans and none that are higher-priced
- institutions with >=100 manufactured loans and none that are higher-priced
- institutions with at least 100 loans and >50% are higher priced 

In [27]:
#list of public HMDA data tables in PG
lar_tables = ["lar_2004_ffiec", "lar_2005_ffiec", "lar_2006_ffiec", "lar_2007_ffiec", "lar_2008_ffiec", "lar_2009_ffiec"
         ,"lar_2010_ffiec", "lar_2011_ffiec", "lar_2012_ffiec", "lar_2013_ffiec", "lar_2014_ffiec", "lar_2015_ffiec",
         "lar_2016_ffiec"]

#make list of production data tables in PG
prod_tables = []
lar_table = "lar_2018_01_"
for x in range(1,29):
    if len(str(x)) <2:
        x = "0" + str(x)
    else:
        x = str(x)
    prod_tables.append("s3_out."+lar_table+x)

In [ ]:
#institutions with >=250 applications with action<=3 with all applications having a single race
sql="""CREATE TEMP TABLE check_counts(
	arid TEXT
	,lender_rows INTEGER)
ON COMMIT DROP;

INSERT INTO check_counts
SELECT
	CONCAT(agency, rid)
	,COUNT(*)
FROM
	public.lar_2016_ffiec
GROUP BY CONCAT(agency, rid);
	
SELECT CONCAT(agency, rid), COUNT(*)
FROM public.lar_2016_ffiec lar
INNER JOIN check_counts ON arid = CONCAT(lar.agency,lar.rid)
WHERE action_type IN ('1', '2', '3')
GROUP BY CONCAT(agency, rid), app_race_1
HAVING COUNT(*) >= 250 


In [ ]:
#agency, rid, name, prior year app count, count state=00 for institutions w >=500 apps with state=00
sql = """
CREATE TEMP TABLE state_counts(
	arid TEXT
	,state_rows INTEGER)
ON COMMIT DROP;

INSERT INTO state_counts
SELECT
	CONCAT(agency, rid)
	,COUNT(*)
FROM
	{table2}
WHERE state='00'
GROUP BY CONCAT(agency, rid)
HAVING count(*) >=500
ORDER BY count(*);

SELECT arid, state_rows, count(l2.*) AS app_count
FROM state_counts l2
LEFT JOIN {table} l1
ON l2.arid = CONCAT(l1.agency, l1.rid)
GROUP BY arid, state_rows"""

In [ ]:
#institutions with >10 single family, site-built, first (junior?) lien, owner occupied, w/ rate spread >8.5 (hoepa)
sql = """  SELECT year, CONCAT(agency, rid) AS ARID, COUNT(*)
   FROM public.lar_2016_ffiec
   WHERE occupancy = '1' AND property_type='1' AND lien_status='2' AND CAST(rate_spread AS FLOAT) >8.5 AND rate_spread NOT LIKE '%NA%'
   
   GROUP BY year, CONCAT(agency,rid)
   HAVING COUNT(*) > 10
   ORDER BY COUNT(*) DESC"""

In [ ]:
#institutions with >10 single family, site-built, first lien, owner occupied, w/ rate spread >6.5 (hoepa)
sql = """
   SELECT year, CONCAT(agency, rid) AS ARID, COUNT(*)
   FROM public.lar_2016_ffiec
   WHERE occupancy = '1' AND property_type='1' AND lien_status='1' AND CAST(rate_spread AS FLOAT) >6.5 AND rate_spread NOT LIKE '%NA%'
   
   GROUP BY year, CONCAT(agency,rid)
   HAVING COUNT(*) > 10
   ORDER BY COUNT(*) DESC
"""

In [ ]:
# frequency by state
sql = """
SELECT year, state_code, count(*)
FROM public.lar_2016_ffiec
GROUP BY year, state_code
ORDER BY state_code ASC"""

In [ ]:
#denial1 if loanpurpose=3, aciton=3: freq, percent, cumulative percent

sql = """  SELECT year, denial_1
    ,(ROUND(COUNT(denial_1)) *100.0/ (SELECT COUNT(*) FROM public.lar_2016_ffiec WHERE action_type='3' AND loan_type='1'),2 ) AS freq_pct
    ,COUNT(*)
    FROM public.lar_2016_ffiec
    WHERE loan_purpose='3' AND action_type='3'
    GROUP BY year, denial_1"""

In [ ]:
#denial1 if loantype=2, action=3: freq, percent, cumulative percent
sql = """
SELECT year, denial_1
    ,(ROUND(COUNT(denial_1)) *100.0/ (SELECT COUNT(*) FROM public.lar_2016_ffiec WHERE action='3' AND loan_type='1'),2 ) AS freq_pct
    FROM public.lar_2016_ffiec
    WHERE loan_type='2' AND action='3'
    GROUP BY year, denial_1
"""

In [ ]:
#denial1 if loantype=1, action=3: freq, percent, cumulative percent

sql = """
    SELECT year, denial_1
    ,(ROUND(COUNT(denial_1)) *100.0/ (SELECT COUNT(*) FROM public.lar_2016_ffiec WHERE action='3' AND loan_type='1'),2 ) AS freq_pct
    FROM public.lar_2016_ffiec
    WHERE loan_type='1' AND action='3'
    GROUP BY year, denial_1
"""

In [ ]:
#inst name for lien=1, loantype=1, propertytype=2, spread>=15: frequency, percent
sql = """ SELECT year, CONCAT(agency, rid)
    ,(ROUND(COUNT(CONCAT(agency, rid)) *100.0/ (SELECT COUNT(*) FROM public.lar_2016_ffiec) )) AS freq_pct
    ,COUNT(CONCAT(agency, rid))

    FROM public.lar_2016_ffiec
    WHERE loan_type='1' AND lien = '1' AND property_type = '2' AND CAST(rate_spread AS FLOAT) >= 15 AND rate_spread NOT LIKE '%NA%'
    GROUP BY year, CONCAT(agency, rid)"""

In [ ]:
#inst name for lien=1, loantype=1, propertytype=1, spread>=15: frequency, percent
sql = """
    SELECT year, CONCAT(agency, rid)
    ,(ROUND(COUNT(CONCAT(agency, rid)) *100.0/ (SELECT COUNT(*) FROM public.lar_2016_ffiec) )) AS freq_pct
    ,COUNT(CONCAT(agency, rid))

    FROM public.lar_2016_ffiec
    WHERE loan_type='1' AND lien_status = '1' AND property_type = '1' AND CAST(rate_spread AS FLOAT) >= 15 AND rate_spread NOT LIKE '%NA%'
    GROUP BY year, CONCAT(agency, rid)
"""

In [ ]:
#rate spread precentiles, mean, stdev, skew, kurtosis


In [ ]:
#purchaser type for loan_type>1, aciton=1: count, percent
sql = """
    SELECT year, purchaser
    ,(ROUND(COUNT(purchaser) *100.0/ (SELECT COUNT(*) FROM lar_2016_ffiec) )) AS freq_pct
    FROM lar_2016_ffiec
    WHERE loan_type!='1'
    GROUP BY year, purchaser
    ORDER BY purchaser"""

In [ ]:
#purchaser type for loan_type=1, aciton=1: count, percent
sql=""" SELECT year, purchaser
    ,(ROUND(COUNT(purchaser) *100.0/ (SELECT COUNT(*) FROM {table}) )) AS freq_pct
    FROM {table}
    WHERE loan_type='1'
    GROUP BY year, purchaser
    ORDER BY purchaser"""

In [ ]:
#income for action=1: percentiles, mean, stdev, skew, kurtosis
#income for action=3: percentiles, mean, stdev, skew, kurtosis
#income for action=1, loan_type=1: percentiles, mean, stdev, skew, kurtosis
#income for action=1, loan_type=2: percentiles, mean, stdev, skew, kurtosis

In [3]:
#ethnicity if race=5: frequency, percent, cumulative percent
sql = """SELECT year, app_eth
    ,(ROUND(COUNT(app_eth) *100.0/ (SELECT COUNT(*) FROM {table}) )) AS freq_pct
 
    FROM {table}
    WHERE race_1='5'
    GROUP BY year, app_eth
    ORDER BY app_eth"""

In [ ]:
#race: frequency, percent, cumulative percent
sql="""SELECT year, race_1
    ,(ROUND(COUNT(race_1) *100.0/ (SELECT COUNT(*) FROM {table}) )) AS freq_pct
    FROM {table}
    GROUP BY year, race_1
    ORDER BY race_1"""

In [ ]:
#co applicant co_sex: frequency percent, cumulative percent
sql="""  SELECT year, co_app_sex
    ,(ROUND(COUNT(co_app_sex) *100.0/ (SELECT COUNT(*) FROM {table}) )) AS freq_pct
    FROM {table}
    GROUP BY year, co_app_sex
    ORDER BY co_app_sex"""

In [ ]:
#applicant sex frequency, percent, cumulative percent

sql="""    SELECT year, app_sex
    ,(ROUND(COUNT(app_sex) *100.0/ (SELECT COUNT(*) FROM {table}) )) AS freq_pct
    ,app_sex
    FROM {table}
    GROUP BY year, app_sex
    ORDER BY app_sex"""

In [1]:
#Race/Action matrix counts
#Could group by MSA and filter lenders for highest delta from race pattern in MSA
sql = """
SELECT * 
FROM crosstab( 
'SELECT action_taken_type, race1, count(race_1)
from {table} group by 1,2 order by 1,2')  
AS final_result(
"action_type" varchar, 
"native" bigint, "asian" bigint, "black" bigint, "islander" bigint, "white" bigint, "no_info" bigint
,"NA" bigint)"""

In [28]:
#action frequency, percent, cumulative percent
def get_action_frequency(table):
    sql = """
     SELECT year, action
    ,(ROUND(COUNT(action) *100.0/ (SELECT COUNT(*) FROM {table}) )) AS freq_pct
    FROM {table}
    GROUP BY year, action
    ORDER BY action""".format(table=table)
    pg_conn.execute(sql)
    colnames = [desc[0] for desc in pg_conn.description]
    data_df = pd.DataFrame(pg_conn.fetchall(), columns=colnames)
    return data_df

first = True
pg_conn = connect()
for table in prod_tables:
    action_df = get_action_frequency(table)
    print(action_df.head())
    if first:
        first = False
        action_freq_df = action_df.copy()
    else:
        action_freq_df = pd.concat([action_freq_df, action_df])
        
pg_conn.close()


   year action_taken_type freq_pct  action_count
0  2018                 1      100           400
   year action_taken_type freq_pct  action_count
0  2018                 1       94           556
1  2018                 3        5            30
2  2018                 4        1             6
   year action_taken_type freq_pct  action_count
0  2018                 1       90          1498
1  2018                 3        4            71
2  2018                 4        5            88
3  2018                 5        1            11
   year action_taken_type freq_pct  action_count
0  2018                 1       81          9829
1  2018                 2        4           521
2  2018                 3        8           955
3  2018                 4        5           667
4  2018                 5        0            39
   year action_taken_type freq_pct  action_count
0  2018                 1       79         10488
1  2018                 2        4           567
2  2018             

In [ ]:
#income_metrics = get_lar_stats("income", lar_tables)
sf_purch_conv_occ_first = "WHERE property_type='1' AND purpose='1' AND loan_type='1' AND occupancy='1' AND lien_status='1'"
sf_purch_conv_occ_junior = "WHERE property_type='1' AND purpose='1' AND loan_type='1' AND occupancy='1' AND lien_status='2'"


s_fam_amount = get_lar_stats("amount", prod_tables, 
     "WHERE property_type='1' AND purpose='1' AND loan_type='1' AND occupancy='1' AND lien_status='1'")
s_fam_amount_df = pd.DataFrame(s_fam_amount)
s_fam_amount_df

#NOTE income analysis needs to handle 'NA   '. This can be addressed on the SQL table by stripping extra white space.
#write code to standardize all NA values to'NA'


In [4]:
#Race by Action taken
            # - count
            # - percentage of total

            #Check metrics by summing components
            #check top 10-20 lenders along these metrics to see if big changes happen
            #cut by geography, lender
#action frequency, percent, cumulative percent


In [5]:
s_fam_inc = get_lar_stats("income", table_list=prod_tables, where="WHERE property_type='1' AND income NOT LIKE '%NA%'")

pulling metrics for: s3_out.lar_2018_01_17
pulling metrics for: s3_out.lar_2018_01_18
pulling metrics for: s3_out.lar_2018_01_19
pulling metrics for: s3_out.lar_2018_01_20
pulling metrics for: s3_out.lar_2018_01_21
pulling metrics for: s3_out.lar_2018_01_22
pulling metrics for: s3_out.lar_2018_01_23
pulling metrics for: s3_out.lar_2018_01_24
pulling metrics for: s3_out.lar_2018_01_25
